In [1]:
import pandas as pd
import numpy as np
from solar.time_handling import add_months
from solar.feature_selection_methods import *
from boruta import BorutaPy
from sklearn.preprocessing import KBinsDiscretizer
from tqdm import tqdm_notebook as tqdm
import sys
if not sys.warnoptions:
    import warnings

ModuleNotFoundError: No module named 'scipy'

In [ ]:
input_df = pd.read_csv('TIMESERIES_all_monthly_enriched_no_dairyntel.csv').ffill()
input_df['date'] = pd.to_datetime(input_df.apply(lambda x: x['date'][:10],axis=1))

In [ ]:
input_df.shape

In [ ]:
def add_features(input_df, time_horizon):
    df = input_df.drop(columns=['date'])
    # stationary
    s_df = df.diff(time_horizon)
    a = s_df.columns.tolist()
    b = [f'STATIONARY_{feature}' for feature in a]
    s_df = s_df.rename(columns=dict(zip(a,b)))
    # Binary
    b_df = np.sign(df.diff(time_horizon))
    b = [f'BINARY_{feature}' for feature in a]
    b_df = b_df.rename(columns=dict(zip(a,b)))
    return pd.concat([input_df, s_df, b_df], axis=1)

In [ ]:
input_df = pd.read_csv('TIMESERIES_all_monthly_enriched_no_dairyntel.csv').ffill()
input_df['date'] = pd.to_datetime(input_df.apply(lambda x: x['date'][:10],axis=1))
time_horizon = 8

In [ ]:
output_df = add_features(input_df,8)
feature_df = output_df.copy()
feature_df['Target'] = -(feature_df['ZuivelNL - Dutch butter quotation'].diff(-time_horizon))
feature_df = feature_df.iloc[:-time_horizon]

model = 'abs_ordinal_regression'
if model == 'abs_ordinal_regression':
    est = KBinsDiscretizer(n_bins= 5 , encode='ordinal', strategy='uniform')
    feature_df['target'] = est.fit_transform(np.abs((feature_df[['Target']])))
else:
    feature_df['target'] = feature_df['Target']

feature_df = feature_df.drop(columns=['Target'])
#feature_df = feature_df.drop(columns=[col for col in feature_df.columns if col.startswith('STATIONARY_')])
input_feat = feature_df.columns.tolist()
input_feat.remove('target')
input_feat.remove('date')

v_feats = remove_invalid_features(feature_df)
v_feats = eliminate_multicolinear_features(feature_df,v_feats)
print(len(v_feats))
print

In [ ]:
print(v_feats)